In [402]:
# Imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
import requests
import re

In [409]:
def get_movies():
    pages = range(1,11)
    
    # Lists of all movie values available
    movie_titles = []
    movie_ids =  []
    release_years = []
    ratings = []
    summaries = []
    directors = []
    actors = []
    certificates = []
    runtimes = []
    genres = []
    lifetime_grosses = []
    
    for page in pages:
        url = f"https://www.imdb.com/list/ls098063263/?sort=list_order,asc&st_dt=&mode=detail&page={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")


        movie_divs = soup.find_all("div", {"class": "lister-item mode-detail"})


        for div in movie_divs:
            #Header - Title, ID and Release year
            header = div.find("h3", {"class": "lister-item-header"})

            # Parser for the header html
            parse_header = BeautifulSoup(str(header), 'html.parser')

            # Get titles
            title = parse_header.find('a').text
            movie_titles.append(title)

            # Get ID
            title_tag = str(parse_header.find('a'))


            movie_id = title_tag.split('/title/')[1].split('/')[0]

            movie_ids.append(movie_id)

            # Get release years
            year = parse_header.find('span', {'class': 'lister-item-year text-muted unbold'}).text

            # Remove (I) and parenthesis
            year = year.replace("(I)", "").replace("(", "").replace(")", "").strip()

            release_years.append(year)

            p1 = div.find_all("p", {"class": "text-muted text-small"})[0]
            p2 = div.find_all("p", {"class": "text-muted text-small"})[1]
            p3 = div.find_all("p", {"class": "text-muted text-small"})[2]

            # Remove all tags
            cleaned_string_p1 = re.sub('<[^>]*>|\\|', '', str(p1))
            cleaned_string_p2 = re.sub('<[^>]*>|\\|', '', str(p2))

            # Rating
            rating_div = div.find("div", {"class": "ipl-rating-star small"})

            # Parser for the rating html
            parse_rating_div = BeautifulSoup(str(rating_div), 'html.parser')

            #Get rating
            rating = parse_rating_div.find('span', {'class': 'ipl-rating-star__rating'}).text
            ratings.append(rating)

            # Summary
            summary = div.find_all("p", {"class": ""})[0]
            summary = re.sub('<[^>]*>|\\|', '', str(summary)).replace('\n', '')
            summaries.append(summary)

            # Lifetime Gross
            lifetime_gross = div.find_all("p", {"class": ""})[1].find_all('b')
            lifetime_gross = re.sub('<[^>]*>|\\|', '', str(lifetime_gross)).replace('\n', '').replace('[', '').replace(']', '').replace('$', '').replace(',', '')

            lifetime_grosses.append(int(lifetime_gross))

            # p1 - Certificates, Runtime and Generes
            cleaned_string_p1 = cleaned_string_p1.replace('\n', ';')
            fields = cleaned_string_p1.split(';')
            cleaned_fields = [field.strip() for field in fields if field.strip() != '']

            certificate = ""
            runtime = ""
            genre = ""

            if(len(cleaned_fields) == 2):
                certificate = 'Not Rated'
                runtime = cleaned_fields[0]
                genre = cleaned_fields[1]
            else:
                certificate = cleaned_fields[0]
                runtime = cleaned_fields[1]
                genre = cleaned_fields[2]

            p1_dict = {
            "certificate": certificate,
            "runtime": runtime,
            "genre": genre    
            }

            # Add certificates, runtimes and genres
            certificates.append(p1_dict['certificate'])
            runtimes.append(p1_dict['runtime'])
            genres.append(p1_dict['genre'])


            #p2 - Directors and Stars

                # Directors
            if('Director' in cleaned_string_p2 and 'Directors' not in cleaned_string_p2):
                start_index = cleaned_string_p2.index('Director:') + len('Director:')
                end_index = cleaned_string_p2.index('Stars:')
                result_string = cleaned_string_p2[start_index:end_index].strip()
                result_string = result_string.replace('\n', '')
                directors.append(result_string)


            if('Directors' in cleaned_string_p2):
                start_index = cleaned_string_p2.index('Directors:') + len('Directors:')
                end_index = cleaned_string_p2.index('Stars:')
                result_string = cleaned_string_p2[start_index:end_index].strip()
                result_string = result_string.replace('\n', '')
                directors.append(result_string)

            # Stars    
            start_index = cleaned_string_p2.index('Stars:') + len('Stars:')
            result_string = cleaned_string_p2[start_index:].strip()
            result_string = result_string.replace('\n', '')
            actors.append(result_string)
    
    data = {'id': movie_ids, 'title': movie_titles, 'release year': release_years, "certificate": certificates, "runtime": runtimes, "genre": genres, "rating": ratings, "summary": summaries, "directors": directors, "actors": actors, "lifetime gross in $": lifetime_grosses}
    df = pd.DataFrame(data)
    return df

In [407]:
get_movies()

,id,title,release year,certificate,runtime,genre,rating,summary,directors,actors,lifetime gross in $
0,tt0499549,Avatar,2009,11,162 min,"Action, Adventure, Fantasy",7.9,A paraplegic Marine dispatched to the moon Pan...,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",2922917914
1,tt4154796,Avengers: Endgame,2019,11,181 min,"Action, Adventure, Drama",8.4,After the devastating events of Avengers: Infi...,"Anthony Russo, Joe Russo","Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",2797501328
2,tt0120338,Titanic,1997,PG-13,194 min,"Drama, Romance",7.9,A seventeen-year-old aristocrat falls in love ...,James Cameron,"Leonardo DiCaprio, Kate Winslet, Billy Zane, K...",2201647264
3,tt2488496,Star Wars: The Force Awakens,2015,11,138 min,"Action, Adventure, Sci-Fi",7.8,"As a new threat to the galaxy rises, Rey, a de...",J.J. Abrams,"Daisy Ridley, John Boyega, Oscar Isaac, Domhna...",2069521700
4,tt4154756,Avengers: Infinity War,2018,PG-13,149 min,"Action, Adventure, Sci-Fi",8.4,The Avengers and their allies must be willing ...,"Anthony Russo, Joe Russo","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",2048359754
...,...,...,...,...,...,...,...,...,...,...,...
995,tt1536044,Paranormal Activity 2,2010,R,91 min,"Horror, Mystery",5.7,After experiencing what they think are a serie...,Tod Williams,"Katie Featherston, Micah Sloat, Molly Ephraim,...",177512032
996,tt0251127,How to Lose a Guy in 10 Days,2003,A,116 min,"Comedy, Romance",6.4,Benjamin Barry is an advertising executive and...,Donald Petrie,"Kate Hudson, Matthew McConaughey, Adam Goldber...",177502387
997,tt0370263,Alien vs. Predator,2004,11,101 min,"Action, Adventure, Horror",5.6,During an archaeological expedition on Bouvetø...,Paul W.S. Anderson,"Sanaa Lathan, Lance Henriksen, Raoul Bova, Ewe...",177427090
998,tt0299977,Hero,2002,PG-13,120 min,"Action, Adventure, Drama",7.9,"A defense officer, Nameless, was summoned by t...",Yimou Zhang,"Jet Li, Tony Leung Chiu-wai, Maggie Cheung, Zi...",177395557


In [410]:
# Save to CSV
get_movies().to_csv('movies.csv', index=False)